In [2]:
import sys

import os
os.environ["NUMPY_EXPERIMENTAL_ARRAY_FUNCTION"] = "1"
import numpy as np
import cupy as cp
import tqdm as tqdm
from numpy.random import randint
import pandas as pd

import warnings
from env import Environment

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
%matplotlib inline

In [3]:
ENV_SIZE = (300, 300, 1250) # Size of environment
TURNS = 2000 # Number of turns
AGENT_BIRTHS_PER_TURN = 100 # Number of agents born each turn

RATIO_RANDOM_BIRTH = 0.1 # Ratio of agent birth locations (i.e. base of environment vs. random)

TRACKED_AGENT_RATIO = 1. # Ratio of agents tracked
MOVE_PREFERENCE_MATRIX = np.array([
    [
        [2, 2, 2],
        [1, 1, 1],
        [np.nan, np.nan, np.nan],
    ],
    [
        [2, 3, 2],
        [1, np.nan, 1],
        [np.nan, np.nan, np.nan],
    ],
    [
        [2, 2, 2],
        [1, 1, 1],
        [np.nan, np.nan, np.nan],
    ]
]) # The agent decides which space to move to by adding this move preference array to the value array of the surrounding environment.

MOVE_PROBABILITY_MATRIX = np.array([
    [
        [0.075, 0.075, 0.075],
        [0.0375, 0.0375, 0.0375],
        [0, 0, 0],
    ],
    [
        [0.075, 0.1, 0.075],
        [0.0375, 0, 0.0375],
        [0, 0, 0],
    ],
    [
        [0.075, 0.075, 0.075],
        [0.0375, 0.0375, 0.0375],
        [0, 0, 0],
    ]
]) # 10% of the time the agent moves randomly to an adjacent space. It is the move probability matrix.


# We want to have XYZ format for all matrices, so satisfy this condition
MOVE_PREFERENCE_MATRIX = np.moveaxis(MOVE_PREFERENCE_MATRIX, (0, 1, 2), (1, 2, 0))
MOVE_PROBABILITY_MATRIX = np.moveaxis(MOVE_PROBABILITY_MATRIX, (0, 1, 2), (1, 2, 0))


SEED = 1000


In [4]:
%%time
ENV_NAME = "vol1"
ENV_ARR = np.load(f"./envs/{ENV_NAME}_environment.npy")
ACC_ARR = np.load(f"./envs/{ENV_NAME}_accumulations.npy")


CPU times: user 3.73 ms, sys: 50.8 ms, total: 54.5 ms
Wall time: 54.4 ms


In [5]:
%%time

env = Environment(ENV_ARR, seed=SEED*2)

for i in tqdm.tqdm(range(TURNS)):
    env.born(AGENT_BIRTHS_PER_TURN, 0)
    env.move(MOVE_PREFERENCE_MATRIX, MOVE_PROBABILITY_MATRIX)

# 36047648.17484999


 17%|█▋        | 337/2000 [00:02<00:12, 133.50it/s]


KeyboardInterrupt: 

In [5]:
"""Show result in competition format"""
env.pd_result.head()

,x,y,z,is_live
0,0,1,801,False
1,0,3,781,False
2,0,238,1001,False
3,298,0,255,True
4,0,2,780,False


In [6]:
"""
Save tracking data. The format for this file is an array with the shape (200000, 2000, 3),
which map to these parameters (Agent, Turn, Position).
"""
np.save('tracking.npy', env.tracking)